# Group Number:

# Student 1: Ryan Meghoe

# Student 2: Nikita Jain

# Student 3: Andrei Rykov

# Downloading Data and Preliminaries

In [1]:
import pickle
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

from zipfile import ZipFile
import requests
import io

In [2]:
def load_zip(url):
    response = requests.get(url)
    response.raise_for_status()
    zipf = ZipFile(io.BytesIO(response.content))
    return {name: zipf.read(name) for name in zipf.namelist()}

def load_array(zipfile, fn):
    return np.load(io.BytesIO(zipfile[fn]))

In [3]:
"""
This cell loads the training, validation or test data as numpy arrays,
with the positions, initial velocities and charge data of the particles.

The position arrays are shaped as
[simulation id, time point (corresponding to t = 0, 0.5, 1 or 1.5), x/y spatial dimension, particle id].

The initial velocity arrays are shaped as
[simulation id, 1 (corresponding to t=0), x/y spatial dimension, particle id].

The charge arrays are shaped as [simulation id, particle id, 1]

"""

data = load_zip('https://surfdrive.surf.nl/files/index.php/s/OIgda2ZRG8v0eqB/download')

features = ['positions', 'velocities', 'charges']
    
positions_train, velocities_train, charges_train = (load_array(data, f'data/train/{f}.npy') for f in features)
positions_valid, velocities_valid, charges_valid = (load_array(data, f'data/valid/{f}.npy') for f in features)
positions_test, velocities_test, charges_test = (load_array(data, f'data/test/{f}.npy') for f in features)

print('Shapes of the training data:\n')
print(f'positions: {positions_train.shape}')
print(f'velocities: {velocities_train.shape}')
print(f'charges: {charges_train.shape}')

Shapes of the training data:

positions: (10000, 4, 2, 5)
velocities: (10000, 1, 2, 5)
charges: (10000, 5, 1)


In [ ]:
print('An example of retrieving data from the arrays:\n\n')

sim_idx = 42
t_idx = 2  # t_idx 0, 1, 2, 3 corresponds to t=0, 0.5, 1 and 1.5 respectively
spatial_idx = (0,1)  # corresponds to both x and y dimension
particle_idx = 3  # corresponds to particle with index 3

p = positions_train[sim_idx, t_idx, spatial_idx, particle_idx]
v = velocities_train[sim_idx, 0, spatial_idx, particle_idx]  # note: this array contains only the inital velocity -> hence the 0
c = charges_train[sim_idx, particle_idx, 0] 

print(
    f'In simulation {sim_idx} of the training set, particle {particle_idx} with charge {c} had coordinates {p}.\nThe initial velocity of this particle was {v}.'
)

In [ ]:
print('Overview of no. datapoints:\n')

print(f'{len(positions_train)} train, {len(positions_valid)} validation, {len(positions_test)} test simulations')

In [ ]:
def plot_example(pos, vel):

    fig = plt.figure()
    axes = plt.gca()
    axes.set_xlim([-5., 5.])
    axes.set_ylim([-5., 5.])
    colors = ['red', 'blue', 'green', 'orange', 'brown']
    for i in range(pos.shape[-1]):
        plt.plot(pos[0, 0, i], pos[0, 1, i], 'd', color=colors[i])
        plt.plot(pos[-1, 0, i], pos[-1, 1, i], 'x', color=colors[i])
        plt.plot([pos[0, 0, i], pos[0, 0, i] + vel[0, 0, i]], [pos[0, 1, i], pos[0, 1, i] + vel[0, 1, i]], '--', color=colors[i])
    fig.set_size_inches(7, 7)
    plt.xlim(np.min(pos)-1, np.max(pos) +1)
    plt.ylim(np.min(pos)-1, np.max(pos) +1)
    plt.plot([], [], 'd', color='black', label='initial position')
    plt.plot([], [], 'x', color='black', label='final position')
    plt.plot([], [], '--', color='black', label='initial velocity \ndirection and magnitude')
    plt.legend()
    
    plt.show()
    return

In [ ]:
random_idx = np.random.randint(0, 10000)
plot_example(positions_train[random_idx], velocities_train[random_idx])

# Data Handling and Preprocessing

In [4]:
import torch

X_train = torch.cat((torch.tensor(positions_train[:,0,:,:]), torch.tensor(charges_train).squeeze(-1).unsqueeze(1)), dim=1)
X_train = torch.cat((X_train, torch.tensor(velocities_train).squeeze(1)), dim=1) # shape: (simulation id, parameters (x, y, c, v_x, v_y), particle id)
y_train = torch.tensor(positions_train[:,1:,:,:]) # shape: (simulation id, time (0.5, 1, 1.5), (x, y), particle id)

X_valid = torch.cat((torch.tensor(positions_valid[:,0,:,:]), torch.tensor(charges_valid).squeeze(-1).unsqueeze(1)), dim=1)
X_valid = torch.cat((X_valid, torch.tensor(velocities_valid).squeeze(1)), dim=1)
y_valid = torch.tensor(positions_valid[:,1:,:,:])

X_test = torch.cat((torch.tensor(positions_test[:,0,:,:]), torch.tensor(charges_test).squeeze(-1).unsqueeze(1)), dim=1)
X_test = torch.cat((X_test, torch.tensor(velocities_test).squeeze(1)), dim=1)
y_test = torch.tensor(positions_test[:,1:,:,:])

In [122]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train, y_train)
valid_dataset = TensorDataset(X_valid, y_valid)
test_dataset = TensorDataset(X_test, y_test)

In [123]:
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model Implementation

In [163]:
import torch.nn as nn
import torch.nn.functional as F

class ParticleModel(torch.nn.Module):
    def __init__(self, set_size: int = 5, input_size: int = 5, 
                 fau1_out: int = 32, gamma1_out: int = 32, fau2_out: int = 32, gamma2_out: int = 2):
        super(ParticleModel, self).__init__()
        # first iteration 
        # layer 1
        # input target (x_t, y_t, c_t, v_t) + neighbour (x_n, y_n, c_n, v_n) + edge (distance(t, n)) 
        # as the result input for first layer is x + x + 1 = 2x + 1 (8 + 1 in our case)
        self.fau_iteration1 = nn.Sequential(nn.Linear(input_size*2 + 1, fau1_out),
                                            nn.ReLU())
        # embeddings are calculated

        # layer 2
        # input (x_target, y_target, c_target, v_target) + embedding from layer_1 + time
        self.gamma_iteration1 = nn.Sequential(nn.Linear(fau1_out+input_size+1, gamma1_out),
                                              nn.ReLU())
        self.embedding_size = (set_size, gamma1_out)

        # second iteration 
        # layer 3
        # input target (embedding from layer 1) + neigbour (embedding from layer 1) + edge (distance(t, n))
        self.fau_iteration2 = nn.Sequential(nn.Linear(gamma1_out*2 + 1, fau2_out),
                                            nn.ReLU())
        # embeddings are calculated

        # layer 4
        # input (x_target, y_target, c_target, v_target) + embedding from layer_3 + time
        self.gamma_iteration2 = nn.Sequential(nn.Linear(fau2_out + gamma1_out + 1, gamma2_out))
        self.output_size = (set_size, gamma2_out)

    
    def forward_iteration1(self, particle_set, distances, target_time: float = 2.):
        embedding = torch.zeros((particle_set.shape[0], self.embedding_size[0], self.embedding_size[1]))
        
        for i in range(particle_set.shape[1]):
            # concatenate the neighborhood
            x = torch.cat([particle_set[:,i].reshape((16, 1, 5)).repeat((1, particle_set.shape[1] - 1, 1)),
                           particle_set[:, list(set(range(particle_set.shape[1])).difference({i}))],
                           distances[:, i, list(set(range(particle_set.shape[1])).difference({i}))].reshape((particle_set.shape[0],\
                                                                                                   particle_set.shape[1]-1,1))],
                          2)


            x = self.fau_iteration1(x)

            # aggregation function
            # mean as a placeholder for now
            x = x.mean(axis = 1)
            # concatenate 
            x = torch.cat([particle_set[:, i].view(particle_set.shape[0], 1, -1),
                           x.view(particle_set.shape[0], 1, -1),
                           torch.Tensor([target_time]).repeat((particle_set.shape[0])).view(particle_set.shape[0], 1, -1)],
                           dim = 2)
            embedding[:,i] = self.gamma_iteration1(x).view(embedding[:,i].shape)
        return embedding

    def forward_iteration2(self, particle_set, distances, target_time: float = 2.):
        embedding = torch.zeros((particle_set.shape[0], self.output_size[0], self.output_size[1]))
        for i in range(particle_set.shape[1]):
            # concatenate the neighborhood
            x = torch.cat([particle_set[:,i].reshape((particle_set.shape[0], 1, particle_set.shape[2])).repeat((1, particle_set.shape[1] - 1, 1)),
                           particle_set[:, list(set(range(particle_set.shape[1])).difference({i}))],
                           distances[:, i, list(set(range(particle_set.shape[1])).difference({i}))].reshape((particle_set.shape[0],\
                                                                                                   particle_set.shape[1]-1,1))],
                          2)

            x = self.fau_iteration2(x)

            # aggregation function
            # mean as a placeholder for now
            x = x.mean(axis = 1)

            
            # concatenate 
            x = torch.cat([particle_set[:, i].view(particle_set.shape[0], 1, -1),
                           x.view(particle_set.shape[0], 1, -1),
                           torch.Tensor([target_time]).repeat((particle_set.shape[0])).view(particle_set.shape[0], 1, -1)],
                           dim = 2)    

            embedding[:,i] = self.gamma_iteration2(x).view(embedding[:,i].shape)
        return embedding

    def forward(self, particle_set, target_time: float = 2.):
        distances = torch.stack([torch.cdist(x_i, x_i) for x_i in particle_set], dim=0)
        p1 = self.forward_iteration1(particle_set = particle_set, distances = distances, target_time=target_time)
        out = self.forward_iteration2(particle_set=p1, distances=distances, target_time=target_time)

        return out

# Model Training

In [ ]:
# train model

# Evaluation

### Some ideas on the experimental part:

* Try various aggregation functions: Mean, Pooling (Max and Min)
* Compare different target time as an input
* Check different losses
* Regularization to the embeddings (linear layers) (?)

In [ ]:
#todo